<a href="https://colab.research.google.com/github/maryryou/synaxarium/blob/main/Synaxarium_Web_Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np

In [ ]:
URL = 'https://www.copticchurch.net/synaxarium/all/en'

r = requests.get(URL)

In [ ]:
# print(r.content)

In [ ]:
soup = BeautifulSoup(r.content, 'html5lib')
# print(soup.prettify())

In [ ]:
# all_links_clean

In [ ]:
# len(all_links_clean)

In [ ]:
base_url = 'https://www.copticchurch.net'

all_links = []

for a in soup.find_all('a', href=True):
    all_links.append(base_url + a['href'])

In [ ]:
all_links = all_links[19:-3]

In [ ]:
all_titles = []
bs=BeautifulSoup(r.content, 'html5lib')
table_body=bs.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]

    all_titles += (cols)

In [ ]:
# all_titles

In [ ]:
months = ['Tout', 'Baba', 'Hator', 'Kiahk', 'Toba', 'Amshir', 'Baramhat', 'Baramouda', 'Bashons', 'Paona', 'Abib', 'Mesra', 'Nasie']

In [ ]:
for i in range(len(all_titles)):
  all_titles[i] = all_titles[i].strip()

In [ ]:
len(all_titles)/2

In [ ]:
list_synax_months = []
list_synax_titles = []

for i in range(len(all_titles)):
  if i%2 ==0:
    list_synax_months.append(all_titles[i])
  else:
      list_synax_titles.append(all_titles[i])

In [ ]:
len(list_synax_months)

In [ ]:
len(list_synax_titles)

In [ ]:
len(all_links)

In [ ]:
data = [list_synax_months, all_links]
columns = ['Month', 'Link']

In [ ]:
df = pd.DataFrame(data)
df = df.T
df.columns = columns


In [ ]:
def remove_period(input_str):
  if len(list(input_str)) > 0 and list(input_str)[-1] == '.':
    # print(input_str)

    input_str = input_str[:-1]

    return input_str

print(len(df))

In [ ]:
r = requests.get('https://www.copticchurch.net/synaxarium/11_25.html?lang=en')

soup = BeautifulSoup(r.content, 'html5lib')

# print(soup.prettify())

# soup.p
# soup.p.find_next_siblings("p")

# print(page)

In [ ]:
import re
def extract_synaxarium_sections(link):

    """
    Extract the full text of the 3 Synaxarium sections from the HTML.

    Args:
        html (str): The HTML content of a Synaxarium page.

    Returns:
        list[str]: List containing the full text of each section.
    """

    r = requests.get(link)

    soup = BeautifulSoup(r.content, 'html5lib')

    html = soup.prettify()

    soup = BeautifulSoup(html, "html.parser")

    headers = soup.find_all("h3")
    sections_dict = {}

    for header in headers:
        title = header.get_text(strip=True)

        content = []
        for sibling in header.next_siblings:
            if sibling.name == "h3":
                break
            if sibling.name or isinstance(sibling, str):
                text = sibling.get_text(" ", strip=True) if hasattr(sibling, 'get_text') else sibling.strip()
                if text:
                    content.append(text)
        sections_dict[title] = " ".join(content).strip()

    return sections_dict

In [ ]:
df['Stories'] = df['Link'].apply(lambda x: (extract_synaxarium_sections(x)))

In [ ]:
df = (
    df.set_index(["Month", "Link"])["Stories"]   # keep id & date as index
      .apply(lambda d: pd.Series(d))       # expand dict into columns
      .stack()                             # reshape into rows
      .reset_index()                       # reset to normal columns
      .rename(columns={"level_2": "Event", 0: "Story"})
)

In [ ]:
df['Link'] = df['Link'].apply(lambda x : x.split('#')[0])

In [ ]:
df.drop_duplicates(inplace = True)
df.reset_index(drop = True)

In [ ]:
df.at[1463, "Story"] = '''This day also marks, the martyrdom of the Saint Abba Isaac, who was from Shama, district of Ashmoun. He worked as a gardener and overseer of an orchard. He was pious, gentle, and devoted to God. He did not eat meat or drink wine. He fasted two days at a time, then broke the fast with legumes. He visited the poor and the needy, and gave them what was left of his wages.

The angel of the Lord appeared to him in a vision and commanded him to go to the governor and confess the Name of Christ to receive the crown of martyrdom. Abba Isaac rejoiced, distributed all that he had, went to the governor, and confessed the Lord Christ. The governor tortured him much, but the Lord strengthened and healed him. Afterwards, the governor cut off his holy head and he received the crown of martyrdom. The people of his hometown came and took his pure body with great honor. The Lord wrought many great signs and wonders of his holy body.

May his prayers be with us. Amen.'''

In [ ]:
df.at[1464, "Story"] = '''On this day also, was the martyrdom of St. Hilaria (Liyarya). She was born in the city of Demliana which is near by the city of Demera, to pious Christian parents. She was chaste, and she fasted and prayed continually. When she was twelve years old, the angel of the Lord appeared to her while she was working and told her, "Why are you sitting here, and the strife is going on and the crown of martyrdom is prepared for you."

She gave all that she had to the poor and departed to Toah, and then to the city of Sarsana, district of El-Shouhada, Menoufiah. She located the governor and confessed the Lord Christ before him. He tortured her much. St. Shenousy was there. He comforted and strengthened her. The governor intensified his torture by making gashes in her body with an iron comb, and he drove red-hot iron pegs into her ears. Afterwards, he bound her along with seven thousand, six hundred martyrs, took them with him and departed. While they were on the ship, a crocodile jumped from the water and snatched away an only child from his mother, who wept and lamented for her son. This saint had compassion on her, and prayed to the Lord Christ. The crocodile returned the child alive and sound. When they came to Toah, the governor cast St. Hilaria (Liyarya) in the fire which did not harm her. Then they hacked off her members, cut off her head and cast them in the fire, thus she received the crown of martyrdom.

May her prayers be with us. Amen.'''

In [ ]:
df.at[1465, "Story"] = '''This day also marks, the martyrdom of Sts. Thecla and Mouji (Mugi). They were born in the city of Kerakus, and they were brought up by a God-fearing woman, who was a teacher. Once when they were crossing the river they saw the governor torturing the Christians. They were amazed from his hard-heartedness. The angel of the Lord appeared to them, and showed them the glory of the saints. They went to Alexandria, where they confessed the Lord Christ before the governor, who tortured them severely. Then, he cut off the head of St. Mouji. As off St. Thecla, he sent her to the city of Demtaw, where she was martyred, after much torture. Thus they received the crown of martyrdom.

May there prayers be with us. Amen.'''

In [ ]:
df.at[1466, "Story"] = '''5. Today also, St. Antonius (Andona), was martyred. He was born in the city of Beba, to pious and merciful parents. When he heard about the torture of the martyrs, he went to Ansena, and confessed the Lord Christ before the governor. The governor ordered him shut to death with arrows. When no harm came upon him, he sent him bound with St. Abimachus and two other martyrs to Alexandria. The Governor shut up the three in prison and crucified St. Antonius head downwards but he was not harmed. When the governor became weary of torturing him, he sent him to the governor of Farma. There he found St. Mina in prison, and they rejoiced for their meeting. The governor of Farma tortured Antonius, at times by combing his body with iron combs, and at other times by casting him in a cauldron filled with boiling oil, but the Lord strengthened and healed him. Afterwards, they cut off his head and thus he received the crown of martyrdom.

May his prayers be with us. Amen.'''

In [ ]:
df.at[1467, "Story"] = '''On this day also, St. Abakragoun, who was from El-Batanoon, was martyred. He was formerly a thief. Two young men plotted with him to steal. They went to the cell of a monk, and they found him keeping a vigil, praying. They waited until he had finished his prayer and had lain down. Nevertheless, he remained standing praying until they became weary and troubled. At dawn, the old monk went out to the thieves, and when they saw him they bowed down before him, and cast down their swords. He preached to them, taught them, and they became monks under his direction.

St. Abakragoun fought a very great fight, and devoted himself to the ascetic life. The old monk prophesied to him that he would receive the crown of martyrdom for the Name of Christ, and his saying was fulfilled later on.

Six years latter the devil incited persecution against the church, St. Abakragoun bid his spiritual father farewell and took his blessings. He went to Nakios, and confessed the Name of the Lord Christ before the governor, who was appointed by Maximianus Caesar. The governor tortured him much and then took him to Alexandria. Then he was tortured by hanging him on the mast of the ship five times, and each time the ropes broke. Then they placed him in a sack made of hide, and threw him into the sea.

The angel of the Lord brought him out of the water, and commanded him to go to Samanoud. On his way he passed by the city of El-Benwan, and its people recognized him. They brought to him everyone who had an infirmity and they were healed by his prayers.

When he arrived at Samanoud, God wrought on his hands many signs. He raised, by his prayers, the daughter of Justus the Vizier from the dead. Justus, his wife, and all his soldiers, believed, and they all received the crown of martyrdom, and they were numbered, nine hundred thirty-five men. As for St. Abakragoun, they sent him back to Alexandria. After they tortured him with different kinds of tortures, they cut off his head, and he received the crown of martyrdom.

The angel of the Lord appeared to a priest from the city of Menuf, told him about the place of the body of the saint, and the priest went and took it. After the end of the time of persecution, they built a church in his name in the city of El-Batanoon, and placed his body in it.

May his prayers be with us. Amen.'''

df.at[1467, "Event"] = '''6. The Martyrdom of St. Abakragoun.'''

In [ ]:
df.at[1469, "Story"] = '''This day also, marks the martyrdom of St. Domadius El-Souriani (The Syrian). He was brought up in Persia, and learned Astrology, and wished to become Christian. One day he found a Persian monk in the market, whose name was Aughalis, who taught him the way of God and he rejoiced exceedingly. He began to preach to the people of his house, and to teach them what might facilitate them to the path of Faith. Then he was baptized, and became a monk. He performed many great deeds that the brethren were jealous of him. When he felt that, he left them and went to the monastery of St. Sergius. There he lived with a hermit for ten years during which he did not eat any cooked food. Then they ordained him a deacon.

While he was ministering with the hermit priest in the sanctuary, he saw a white dove of most beautiful appearance, which came down and descended upon the altar. Domadius thought that it was a real dove, so he made a motion with his hand to drive the dove off, for he was worried about the content of the chalice. After the service was ended, the priest asked him, "What did disturb you during the  liturgy?" Domadius told him what he had seen. The priest told him, "If you see the dove again tell me."

In the next day, the priest went up to the altar to celebrate the liturgy as usual, and when Domadius saw the dove, he told the priest, "My father the dove has come." The priest looked around but he could not see anything. He bowed his head before God with weeping and prayer by day and night for a period of time, until he was worthy to see this dove. He realized that this was the sign of the Holy Spirit. He did not say anything to St. Domadius lest pride fill his heart, but he told the bishop who had ordained him a priest.

When the news of his holiness reached the patriarch of this area, he wished to visit him. When St. Domadius knew of this, he fled from there and came to the church of St. Cosmas, where he lived for a while nearby, feeding on the vegetation of the earth. God wrought on his hand many signs and miracles.

When Emperor Julian went out to fight against the Persians, he passed by the cave of this saint. They told him about the saint, so he ordered him stoned. The soldiers stoned him and his disciple, until a great heap of stones stood over the cave. Many years later, God revealed his body, a church was built for him and God manifested many wonders in it.

May his prayers be with us. Amen.'''

In [ ]:
df.at[1470, "Story"] = '''Today also the church celebrates the commemoration of the consecration of the church of the great St. Mercurius Abu-Saifain. The biography of his life is written under the twenty- fifth day of the month of Hatour.

May his blessing be with us. Amen.'''

In [ ]:
df.at[1471, "Story"] = '''On this day also, of the year 316 A.M., the saint Anba Palamon the anchorite, departed. He was an anchorite in the Eastern Mount in a town called El-Kasre and El-Sayad, district of Nagaa-Hammady, governorate of Quena. The Devil was weary of tempting this father, and failed to overcome him. The Devil took every chance to direct his snares against him.

One day, the man of God Anba Palamon rose up, took the work of his hand, and walked toward the countryside of Egypt. While he was walking in his way, he was weeping for his sins. The enemy of the good led him astray in the mountain, for seven days, until he was close to death from hunger and thirst, for it was summer. Finally, he collapsed, fell on the ground, wishing to die. God, the lover of mankind, wished not to leave His servant Anba Palamon fall prey to the devil, so He ordered the devil to leave him. When the old man realized that, he cried saying, "O Lord Jesus Christ help me." Instantly he heard a voice saying to him, "Do not be afraid, for the enemy can not overcome you. Rise and walk a short distance to the south. You shall find a righteous old monk whose name is Talasoun. Tell him about all that the devil has brought on you, and the grievous sin that he tempted you with in your young age. He will pray on your behalf so that your sins will be forgiven." The blessed Anba Palamon rose up, carried the works of his hands, and walked in the mount reciting Psalm fifty four, "Save me, O God, by Your name, And vindicate me by Your strength. Hear my prayer, O God; Give ear to the words of my mouth. For strangers have risen up against me, And oppressors have sought after my life; They have not set God before them. Behold, God is my helper; The Lord is with those who uphold my life. He will repay my enemies for their evil. Cut them off in Your truth. I will freely sacrifice to You; I will

praise Your name, O LORD, for it is good. For He has delivered me out of all trouble; And my eye has seen its desire upon my enemies." He also recited Psalms 68, "Let God arise, Let His enemies be scattered; Let those also who hate Him flee before Him. As smoke is driven away, So drive them away; As wax melts before the fire, So let the wicked perish at the presence of God. But let the righteous be glad; Let them rejoice before God; Yes, let them rejoice exceedingly." He did not cease praying until God guided him to the place of Anba Talasoun. When Anba Talasoun saw him, he rejoiced exceedingly, greeted him, held his hand and pulled him up the rock where he lived. They prayed, then sat down talking about the greatness of God. Anba Talasoun asked Anba Palamon about how he found his way to come to visit him in this wilderness.

Then Anba Palamon started crying and kneeled, saying, "Please forgive me O my beloved holy father." Anba Talasoun said, "The Lord Jesus Christ forgives to all of us all our transgressions." Anba Palamon replied saying, "I am ashamed to tell you, O my holy father, about the enormous sin that befell me by the Devil without knowing." Anba Talasoun told him, "It is written 'Confess your faults one to another, and pray one for another, that you may be healed.'"

When Anba Palamon saw that the blessed Anba Talasoun was comforting and encouraging him, he started to tell him, "While I was living in the monastery, I asked God to forgive me my sins. I also read in the holy books of the fathers which are inspired by the Spirit of God, that solitary life brings forth the fear of God, and God despises mockery and laughing for no reason. So I decided to live a solitary life in my small home, to not talk to anyone, and never to laugh, but to cry on my sins by day and night.

The Devil often fought against me. He tried to please my heart with laughter but I did not heed him, and I never cursed. I fought for a long time through these tribulations, steadfast in the salvation of our Lord Christ until the enemy was wrath with me. One day I took the work of my hands, walked in the mount to go to sell it and buy a little of bread. After I was at a considerable distance from my abode, the Devil captivated me, and right away I lost my mind. He removed the Name of God from my mouth and I was no longer reciting the Divine sayings. The mount changed entirely before me, the sandy earth became black. When I looked before me I saw a new city, had splendid buildings, magnificent houses, rich palaces, and the city had shining armored gat which was fortified as a city of a king, surrounded by trees and gardens. When I saw it, I wondered about this city and its great distinction."When I inclined to enter the city thinking that its people might buy from me the work of my hands. As I came close to the walls of the city, I found a turning water wheel (sakieh), and I saw a sorrowful woman. Her face was very gloomy. Her clothes were torn and her veil covered her eyes for the sake of decency. She was standing beside the well, running the water wheel and watering the vines. When she saw me, she sat down, covered her head and said, 'Bless me O my holy father.' In hast she put down the baskets before me then said, 'Sit down and rest O my beloved father for you had become tired from what you have carried.' She sat me down beside running water, she took water in her hands, poured it on my feet, and washed them as someone taking blessings. She acted as a woman of a rich man. I asked her, 'O faithful woman, tell me, if I enter this city with the work of my hands would I find one who would buy it from me?' She said, 'Yes, they would buy it from you. Nevertheless, leave it and I will buy it from you, and I will give you all that you need. For I was married to a rich man who died recently and left me much money and herds of cattle. I pick the fruits of these enormous vines, and I do not have anyone to oversee it. I wish to have a man like you, to whom I can hand all my possessions, to handle the way he sees fit. So if you accept, O my holy father, to come and oversee my house and to take charge of all my possessions, I will take you as a husband.' I replied, 'If the monk gets married, shame and disgrace follow him.' Then the woman told me, 'If you do not take me as a wife, be an overseer for my possessions. You supervise it during the day, for I own fields, cattle, barns, vineyards, slaves and maiden servants, and when the night falls you shall fulfill your prayers.' Then she rose up, and took me to her house. She prepared a table of different kinds of foods and placed it before me. Afterwards, she went to her room, and put on expensive and extravagant clothes, then came and embraced me. I was surprised and I held fast to the power of God almighty, and crossed myself with the sign of the cross. All that I saw immediately vanished as the smoke before the winds, and nothing of what this woman did remained.

"Instantly, I realized that all of this was the doing of the Devil who wished to make me fall in sin. I cried bitterly, and regretted much what I had done. The Lord had compassion on me with His tender mercy, and sent me His angel who comforted me, and promised the forgiveness of my sins and told me, 'Go to the saint Anba Talasoun who is nearby you, and confess to him your sins. Tell him that you have come to him so that the Lord may forgive your sins through his prayers.'" The saint Anba Talasoun prayed saying, "O my son may the Lord forgive you and myself." Subsequently, a table came down from heaven, they ate, then Anba Palamon returned to his place of worship in peace. This Saint received the gift of healing from God. The beasts were friendly to him, he fed them by his hands, and they licked his feet. He lived naked, so God lengthened his hair until it covered his whole body. He fasted the whole week, only ate on Saturday and Sunday. He ate a half loaf of bread that God sent him with the raven. Often he ate from the vegetation of the mount. He was merciful, compassionate, and tender like His creator. When the night fell, he went down from his place of worship, to visit the prisoners, orphans, widowers, those in trouble, and the sojourners, as his strength allowed him, to help them from the price of the works of his hands.

St. Palamon was fervent in his asceticism and worship. He persevered, all the days of his life, in the daily and nightly prayers, and watching all night in ascetic worships. St. Pachomius17, the father of the monastic cenobite, was the disciple of this saint. When St. Pachomius18 rejected paganism and adopted Christianity, he wished to live a life of asceticism and worship, so the priest of his hometown directed him to go to the great hermit Anba Palamon.

Pachomius handed the care for the poor and the needy in his town to another elder monk, and went to Anba Palamon. When he arrived, he knocked the door of his cell. The old man looked from an aperture and asked him, "Who are you O brother? and what do you want?" Pachomius answered hastily, "I, O blessed father, looking for the Christ the God Whom you do worship, and I beg your fatherhood to accept me and to make me a monk." Abba Palamon told him, "O my son, monasticism is not an unrestricted labor, and the man does not come into it as he pleases, for many had adopted it not knowing its hardships, and when they adopted it they could not endure it, and you have heard about it without knowing its contention."

Pachomius replied, "Do not turn my request, and my desire, and do not put off the flame of my zeal. Accept me, be patient with me, and examine me, and afterward do whatever you see fit with me." The old man told him, "Go, O my son test your self alone for a period of time, then come back to me for I am willing to labor with you as much as my weakness allows so you might know your own self. The monastic piety needs toughness and asceticism, and I will teach you first its extent. Then you go and examine your self if you could endure the matter or not. My intent in that, God knows, is for the sake of your teaching and discipline and not for any other reason. When we, O my beloved son, knew the vanity and trickery of this world, we came to this distant place, and carried on our shoulders the cross of our Christ, not the wooden rod, but the overcoming of the flesh, subduing its lusts, and wasting its power. We spent the night praying and glorifying God. Often we vigil from sun set till the morning praying and working much with our hands, making either ropes, braid palm fibers, weave palm leaves or hair to resist sleep and to have what our bodies need and to feed the poor as the apostle said, 'remember the needy.' We entirely do not know the eating of oil, cooked food or drinking liquors. We fast till the evening of the summer days, and two days at a time during the winter then we break the fast by eating only bread and salt. We keep away the boredom by remembering death, and how close it is. We refute every pride and exaltation and guard ourselves from evil thoughts by humility and piety. By this ascetic strive that is accomplished by the grace of God, we offer our souls a living sacrifice, acceptable to God, not only once but many times. For according to the strive and how much we exert ourselves in it, we realize spiritual gifts, remembering the saying of the Lord, "Those overcame themselves, seize the Kingdom of Heaven."

When Pachomius heard from Abba Palamon these sayings that he never heard before, he was more assured by the spirit, and was encouraged on facing the hardships and endure the pains. He replied saying, "I am confident in the Lord Christ first, and by the support of your prayers second, I would be able to fulfill all the precepts, and persevere with you till death."

Pachomius then kneeled before him and kissed his hand. The old man preached him and instructed him about the importance of mortifying the flesh, the humility and contrition of the heart. He told him, "If you kept what I told you, did not go back, or vacillate between two opinions, we will rejoice with you." Then he told him, "Do you think my son that by all what I mentioned to you from asceticism, praying, watching and fasting we ask for the glory of men, no my son, it is not like that. We lead you to the works of salvation so we would be blameless, for it is written every apparent thing is light, and we must through many tribulations enter the kingdom of heaven. Now go back to your abode so you might examine your self and test her for few days for what you are asking is not an easy task." Pachomius answered him saying, "I have examined myself in every thing, and I hope with the Grace of God and your holy prayers, that your heart would be pleased toward me." The old man answered, "Well."

Anba Palamon accepted him with joy, then left him for ten days testing him in prayer, watching and fasting. Three month later and after he tested his patience, endurance, strife and will, he prayed over him, clipped off his hair, and put on him the garb of the monks (El-Eskeem) in year 304 A.D.

They persevered together on asceticism and prayer as they worked in their spare times in weaving hair and knitting clothes to gain their bare necessities, and what they spared they gave to the needy. Whenever they were vigilance and the sleep overcame them, they went out their cells and moved sand from one place to another to exhaust their bodies and remove the sleep away from them. The old man went on preaching the young man and encouraging him saying, "Be courageous O Pachomius, let your devotion to God always be flamed with the fire of love, and be before Him, in fear, humility, and persevering in praying and kneeling without boredom. Be watchful lest the tempter examine you and grief you."

It came in the biography of St. Pachomius in a manuscript in the monastery of El-Baramous the following:

One day someone knocked on the door of Sts. Palamon and Pachomius to visit them, and he spent the night with them. The pride and self-reliance had overcome that person. While they were talking about the words of God and before them was fire, for it was winter, the guest told them, "Whoever of you has strong faith in God, let him rise up, stand on this blazing embers, and recite the prayer which the Lord had taught to His disciples." When the old man St. Palamon heard that, he admonished him saying, "Cursed is the defiled devil who planted this thought in your heart. so stop talking." The guest did not head to the words of the old man and said, "I, I would." He rose up and stood on the blazing coal while saying the Lord's prayer slowly. Then he stepped out the fire which did not touch his body with any harm, and went to his abode with arrogance. Pachomius told Anba Palamon, "God knows that I marveled about this brother who stood on the amber and his feet did not burn." The old man replied saying, "Do not be amazed for there is no doubt that this from the work of the Devil. The Lord had allowed his feet not to be burned as it is written to the crooked, God sends crooked ways. Believe me O my son, if you know the torment that is prepared for him, you will be weeping for his wretchedness."

The Devil came to this man in the form of a woman and enticed him to allow her to enter his cell. Because of the pride and the blindness of his perception, he did not realize the danger that surrounded him. The lust filled his heart toward her, and immediately the devil struck and threw him on the ground and he remained as a dead man for a day. When he regained his conscious and his faculties, he went to Abba Palamon weeping and remorseful for what he had done. He asked for their help and prayers for the devil had captivated him by his own free will. While he was talking to them, the old man and his disciple were weeping, the evil spirit suddenly came over him. He went to the mountain, and he lost his mind. He was astray for a period of time, then he threw himself in a fire which burned him. When the old man knew that, he was grieved. His disciple asked him, "How God allowed this to happen to him after he confessed and asked for repentance with tears and remorse." The Abba answered, "God with His foreknowledge knew that the repentance of this brother was not sincere, so He allowed him to reap what he did."

While St. Pachomius was still staying with his teacher Anba Palamon, one day he was wondering in the wilderness, and he came to the village of Tabanseen. When he was praying, the angel of the Lord appeared and told him, "O Pachomius, with the order of the Lord, build a monastery in the spot that your are standing on, hence many will come to you seeking monasticism." He returned to the old man Anba Palamon and told him what the angel had told him, and his intention of fulfilling the Will of God. Anba Palamon was sad for the departure of his disciple and said, "How could you leave me after seven years you spent with me in obedience and submission and I am an old man. I see that it is easier on me to accompany you than you leaving me."

They moved south until they came to Tabanseen, and started to build a monastery. That was in the year 311 A.D., and Pachomius was thirty years old. When they finished building the monastery, Anba Palamon told his disciple Pachomius, "My beloved son I long to return to my cell and the place of my solitude. I had known that God had appointed you to establish this monastery, which will grow and be filled with God pleasing monks. You shall receive power and long suffering to manage them. As of my self, I have became old, weak, and the time of my departure had drew near. So I see that my solitary life is best for me. Nevertheless, I ask from your kindness that you do not deprive me from seeing you from now and then. I shall come to visit you as the few days left for me allow." They were separated after they prayed with each other, and they visited each other. In one of the visits of Anba Palamon, he became ill and departed to the Lord that he served and loved since his young age. Anba Pachomius took the blessings of his teacher, shrouded and buried him.

Many wonders and miracles have been attributed to this saint. An ancient church in his name is located in the town of El-Kasre and El-Sayad in a monastery named after him which had other churches in the name of the Virgin St. Mary, the archangel Michael, St. Mercurius Abu-Saifain, and St. Demianah. A great festival celebration is celebrated annually in his feast day. May his prayers be with us and Glory be to God forever. Amen.'''

In [ ]:
df = df.drop(1468)
df = df.drop(1690)

In [ ]:
df["Event"] = df["Event"].str.replace(r"^\d+\.\s*", "", regex=True)


In [ ]:
df.to_csv("Synaxarium Project Clean Sep 2025.csv")